In [1]:
import os
import pandas as pd
import maup
import geopandas as gp

import sys; sys.path.append("..") # Adds parent directory to python modules path.
from topdown_parsers import *

In [ ]:
def modify_race(race):
    """
    """
    if race == "w":
        return 1
    elif race == "b":
        return 2
    elif race == "i":
        return 3
    elif race == "a":
        return 4
    elif race == "h":
        return 5
    else:
        return 6

def add_breakdown_cols(df):
    # make bool columns for the counts
    df["race"] = df["race"].map(lambda race: modify_race(race))

    # voting age
    df["VAP"] = df["age"] >= 18
    
    df = df.rename(columns={"ethn": "HISP"})
    
    return df

def get_zero_rows(df):
    """
    """
    zeros_df = pd.DataFrame(columns=["GEOID", "race", "VAP", "HISP", "count"])
    it = 0
    
    geoids = []
    races = []
    vaps = []
    hisps = []
    counts = []
    
    for geoid in df["GEOID"].unique():
        for race in [1, 2, 3, 4, 5, 6]:
            for hisp in [0, 1]:
                for vap in [True, False]:
                    if ((df["GEOID"] == geoid) & 
                        (df["race"] == race) & 
                        (df["VAP"] == vap) & 
                        (df["HISP"] == hisp)).any():
                        continue
                    
                    geoids.append(geoid)
                    races.append(race)
                    vaps.append(vap)
                    hisps.append(hisp)
                    counts.append(0)
    
    zeros_df["GEOID"] = geoids
    zeros_df["race"] = races
    zeros_df["VAP"] = vaps
    zeros_df["HISP"] = hisps
    zeros_df["count"] = count
    
    return zeros_df

In [6]:
texas_df = read_and_process_reconstructed_csvs("./Reconstructed")
texas_df = add_breakdown_cols(texas_df)
texas_breakdown = texas_df.groupby(["county", "HISP", "race", "VAP"]).count().iloc[:,1].reset_index().rename(columns={"age":"count"})

Reading files...
Duplicating rows... (based on reconstructed solutions)
Processing Geo IDs
Splitting into geounits...
Cleaning up the geounits...
Converting to fips codes and paddings them up...
Building Enumdist column
Done with reading and preparing data.


In [3]:
dallas_df = read_and_process_reconstructed_csvs("./Reconstructed/Texas_Dallas County")
dallas_df = add_breakdown_cols(dallas_df)
dallas_breakdown = dallas_df.groupby(["GEOID", "HISP", "race", "VAP"]).count().iloc[:,1].reset_index().rename(columns={"age":"count"})

Reading files...
Duplicating rows... (based on reconstructed solutions)
Processing Geo IDs
Splitting into geounits...
Cleaning up the geounits...
Converting to fips codes and paddings them up...
Building Enumdist column
Done with reading and preparing data.


In [4]:
dallas_breakdown

,GEOID,HISP,race,VAP,count
0,481130001001001,0,1,True,5
1,481130001001004,0,1,False,13
2,481130001001004,0,1,True,82
3,481130001001004,1,6,True,4
4,481130001001006,0,1,False,12
...,...,...,...,...,...
159689,481139801001022,0,4,False,2
159690,481139801001022,0,4,True,2
159691,481139801001042,0,1,True,1
159692,481139801001042,1,1,True,4


In [15]:
texas_breakdown

,county,HISP,race,VAP,count
0,001,0,1,False,3054
1,001,0,1,True,30442
2,001,0,2,False,1241
3,001,0,2,True,9091
4,001,0,3,False,29
...,...,...,...,...,...
5253,507,1,4,True,2
5254,507,1,5,False,6
5255,507,1,5,True,7
5256,507,1,6,False,261


In [19]:
texas_breakdown.to_csv("texas_breakdown.csv", index=False)
dallas_breakdown.to_csv("dallas_breakdown.csv", index=False)